In [0]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
request = drive_service.files().get_media(fileId = '1XQgPGGrzOU24SGcujJOZ9mPpDDfjaV4o').execute()
import io
data = pd.read_csv(io.BytesIO(request))
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [0]:
data.shape

(7009728, 29)

**7009728** quan sát, 29 cột

In [0]:
data.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [0]:
data.describe() # data numerical

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7.009728e+06,6.855029e+06,7.008884e+06,6.855029e+06,6.855029e+06,6.873482e+06,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,2.224200e+03,1.273224e+02,1.288668e+02,1.040186e+02,8.168452e+00,9.972570e+00,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,1.961716e+03,7.018731e+01,6.940974e+01,6.743980e+01,3.850194e+01,3.531127e+01,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,-1.410000e+02,0.000000e+00,-5.190000e+02,-5.340000e+02,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.220000e+02,7.700000e+01,8.000000e+01,5.500000e+01,-1.000000e+01,-4.000000e+00,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,1.571000e+03,1.100000e+02,1.100000e+02,8.600000e+01,-2.000000e+00,-1.000000e+00,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,3.518000e+03,1.570000e+02,1.590000e+02,1.320000e+02,1.200000e+01,8.000000e+00,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01
max,2008.0,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.743000e+03,1.379000e+03,1.435000e+03,1.350000e+03,2.461000e+03,2.467000e+03,4.962000e+03,3.080000e+02,4.290000e+02,1.000000e+00,1.000000e+00,2.436000e+03,1.352000e+03,1.357000e+03,3.920000e+02,1.316000e+03


In [0]:
data.describe(include='O') # data category

,UniqueCarrier,TailNum,Origin,Dest,CancellationCode
count,7009728,6926363,7009728,7009728,137434
unique,20,5373,303,304,4
top,WN,N476HA,ATL,ATL,B
freq,1201754,4701,414513,414521,54904


Category data bao gồm: UniqueCarrier,	TailNum	Origin,	Dest,	CancellationCode, và Year,	Month,	DayofMonth,	DayOfWeek

In [0]:
missing_val_in_column = (data.isnull().sum())
print(missing_val_in_column[missing_val_in_column >0])

DepTime               136246
ArrTime               151649
TailNum                83365
ActualElapsedTime     154699
CRSElapsedTime           844
AirTime               154699
ArrDelay              154699
DepDelay              136246
TaxiIn                151649
TaxiOut               137058
CancellationCode     6872294
CarrierDelay         5484993
WeatherDelay         5484993
NASDelay             5484993
SecurityDelay        5484993
LateAircraftDelay    5484993
dtype: int64


CancellationCode     6872294 <br>
CarrierDelay         5484993<br>
WeatherDelay         5484993<br>
NASDelay             5484993<br>
SecurityDelay        5484993<br>
LateAircraftDelay  5484993<br>

Những features trên thiếu quá nhiều nên sẽ drop<br>

Feature: Year chỉ ra giấu cho năm 2018 nên drop.

### Handling missing values

In [0]:
data = data.drop(columns=['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Year'])
data = data.dropna(subset=['DepTime', 'ArrTime', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
                          'DepDelay', 'TaxiIn', 'TaxiOut'])
# Tạo cột mới với encode cho target column ArrDelay
data['ArrDelay_ecd'] = np.where(data['ArrDelay'] >= 30, 1, 0)
# xóa cột ArrDelay đi vì ta đã có cột ArrDelay_ecd
data = data.drop(columns=['ArrDelay'])

In [0]:
# check missing value again
missing_val_in_column = (data.isnull().sum())
print(missing_val_in_column[missing_val_in_column >0])

Series([], dtype: int64)


Dữ liệu đã hết missing values

In [0]:
data.describe()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,ArrDelay_ecd
count,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6855024.0,6855024.0,6.855024e+06
mean,6.388525e+00,1.573826e+01,3.925802e+00,1.333775e+03,1.325037e+03,1.481265e+03,1.493715e+03,2.212505e+03,1.273224e+02,1.290778e+02,1.040186e+02,9.923807e+00,7.287441e+02,6.860150e+00,1.644368e+01,0.0,0.0,1.352895e-01
std,3.398833e+00,8.801085e+00,1.988459e+00,4.780574e+02,4.643130e+02,5.051322e+02,4.826643e+02,1.956435e+03,7.018733e+01,6.953273e+01,6.743981e+01,3.520373e+01,5.632449e+02,4.931225e+00,1.131062e+01,0.0,0.0,3.420326e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,-1.410000e+02,0.000000e+00,-5.340000e+02,1.100000e+01,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00
25%,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.180000e+02,7.700000e+01,8.000000e+01,5.500000e+01,-4.000000e+00,3.260000e+02,4.000000e+00,1.000000e+01,0.0,0.0,0.000000e+00
50%,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.317000e+03,1.512000e+03,1.515000e+03,1.563000e+03,1.100000e+02,1.110000e+02,8.600000e+01,-1.000000e+00,5.810000e+02,6.000000e+00,1.400000e+01,0.0,0.0,0.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.905000e+03,3.494000e+03,1.570000e+02,1.590000e+02,1.320000e+02,8.000000e+00,9.540000e+02,8.000000e+00,1.900000e+01,0.0,0.0,0.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.359000e+03,9.741000e+03,1.379000e+03,6.600000e+02,1.350000e+03,2.467000e+03,4.962000e+03,3.080000e+02,4.290000e+02,0.0,0.0,1.000000e+00


In [0]:
data = data.drop(columns=['TailNum', 'Origin', 'Dest','DepDelay', 'Cancelled','Diverted'])
data = data.drop(data[data['CRSElapsedTime'] < 0].index, axis=0)

Down data xuống còn 35%

In [0]:
data = data.sample(frac=0.35, random_state=1) # lấy mẫu ngẫu nhiên xuống còn 35%, flag random state = 1

Target column giờ là ArrDelay_ecd

In [0]:
data['ArrDelay_ecd'].value_counts()

0    2074282
1     324973
Name: ArrDelay_ecd, dtype: int64

Số chuyến đến trễ **927403** so với đến sớm **5927611**<br>
Vậy dữ liệu imbalance

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
data.columns

Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime',
       'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'Distance', 'TaxiIn', 'TaxiOut',
       'ArrDelay_ecd'],
      dtype='object')

In [0]:
data.shape

(2399255, 16)

'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier' là category value nên sẽ dùng one hot encoder.

In [0]:
X = data.drop(['ArrDelay_ecd', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'], axis=1)
y = data["ArrDelay_ecd"] # target column

**One hot encoder**

In [0]:
n= data[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]

In [0]:
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
type(one_hot)

pandas.core.frame.DataFrame

In [0]:
type(X)

pandas.core.frame.DataFrame

Giờ ta sẽ nối các cột đã one hot encode vào dataframe X để có các features đầy đủ

In [0]:
new_X = pd.concat([X, one_hot], axis=1, sort=False)
new_X.shape

(2399255, 81)

In [0]:
X.shape

(2399255, 11)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size = 0.2, random_state = 0)

In [0]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.81      0.86    414905
           1       0.29      0.50      0.37     64946

    accuracy                           0.77    479851
   macro avg       0.60      0.65      0.61    479851
weighted avg       0.83      0.77      0.79    479851



In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414905
           1       0.98      0.53      0.69     64946

    accuracy                           0.94    479851
   macro avg       0.96      0.77      0.83    479851
weighted avg       0.94      0.94      0.93    479851



In [0]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    414905
           1       0.96      0.96      0.96     64946

    accuracy                           0.99    479851
   macro avg       0.98      0.98      0.98    479851
weighted avg       0.99      0.99      0.99    479851



In [0]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.96      1.00      0.98    414905
           1       0.97      0.71      0.82     64946

    accuracy                           0.96    479851
   macro avg       0.96      0.85      0.90    479851
weighted avg       0.96      0.96      0.95    479851



In [0]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    414905
           1       0.96      0.51      0.67     64946

    accuracy                           0.93    479851
   macro avg       0.95      0.75      0.81    479851
weighted avg       0.93      0.93      0.92    479851



In [0]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


![alt text](https://machinelearningcoban.com/assets/33_evaluation/PR.png)

- Precision của Gradient Boosting Classifier ở class 1 là 0.96, support 185747<br>
Tức là trong số điểm được phân loại là 1 là **đúng trong những điểm được phân loại là 1** chiếm tỷ lệ **96%**<br>
- Recall của Gradient Boosting Classifier ở class 1 là 0.52<br>
Tức là **tỷ lệ tìm ra các điểm các điểm** class 1 là **52%**<br>
- F1 là harmonic mean

$$ F_1 = 2 \times \frac{precision \times recall}{precision + recall} $$

